# Business challenge/requirement
BookRent is the largest online and offline book rental chain in India. TheCompany charges a fixed fee per month plus rental per book. So,the company makes more money when user rent more books. You as an ML expert have to model recommendation engine so that user gets recommendation of books based on the behaviorof similar users. This will ensure that users are renting books based on their individual taste.Company is still unprofitable and is looking to improve both revenue and profit.

Key issues As of now lot users return book and do not take new rental. Right recommendation will entice user to rent more books

Considerations NONE

Data volume-Approx 1 M records –file BX-Book-Ratings.csv and 2 more. But only 10K records will be used

Fields in Data •user_id: Unique Id of the User •isbn: International Standard Book Number is a unique numeric commercial book identifier •rating: rating given by userAdditional information-NA

In [1]:
import pandas as pd
col_names = ['user_id','book_id','rating','timestamp']
df = pd.read_csv("Recommend.csv",header=None,names=col_names)
df.head()

,user_id,book_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [2]:
n_users = df.user_id.unique().shape[0]
n_books = df.book_id.unique().shape[0]
print(n_users)
print(n_books)

943
1682


In [3]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)

In [4]:
#create train data matrix
import numpy as np
train_data_matrix = np.zeros((n_users,n_books))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]
train_data_matrix

array([[5., 3., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [5]:
#create train data matrix
test_data_matrix = np.zeros((n_users,n_books))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]
test_data_matrix

array([[0., 0., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
#user based filtering
from sklearn.metrics import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric = 'cosine')
mean_user_rating = train_data_matrix.mean(axis=1)[:,np.newaxis]
ratings_diff = (train_data_matrix - mean_user_rating)
user_pred = mean_user_rating + user_similarity.dot(ratings_diff)/np.array([np.abs(user_similarity).sum(axis=1)]).T
user_pred

array([[ 1.61867545,  0.54581833,  0.45811662, ...,  0.28101873,
         0.28115459,  0.27855055],
       [ 1.37273886,  0.25819097,  0.12154629, ..., -0.07659324,
        -0.07530581, -0.07855732],
       [ 1.38281529,  0.22869676,  0.09690448, ..., -0.1063691 ,
        -0.10458322, -0.10783782],
       ...,
       [ 1.28052878,  0.19104009,  0.06107922, ..., -0.12963759,
        -0.12878331, -0.13190331],
       [ 1.40783871,  0.2971023 ,  0.19077243, ..., -0.00471531,
        -0.00405873, -0.00688292],
       [ 1.46637532,  0.37064178,  0.29052286, ...,  0.1162864 ,
         0.11619799,  0.11376516]])

In [8]:
#content based filtering
book_similarity = pairwise_distances(train_data_matrix.T, metric = 'cosine')
book_pred = train_data_matrix.dot(book_similarity) / np.array([np.abs(book_similarity).sum(axis=1)])
book_pred

array([[0.35444393, 0.36768477, 0.3951705 , ..., 0.43253286, 0.42482726,
        0.43010113],
       [0.07950718, 0.08966955, 0.08670188, ..., 0.08986078, 0.09123945,
        0.09101725],
       [0.06053844, 0.06355636, 0.06122776, ..., 0.05916555, 0.06237366,
        0.06246282],
       ...,
       [0.02626312, 0.03137255, 0.03090226, ..., 0.03523371, 0.03451008,
        0.03509816],
       [0.12832898, 0.13707072, 0.14628314, ..., 0.15301357, 0.15134554,
        0.15348007],
       [0.21194668, 0.20512203, 0.23490228, ..., 0.26864792, 0.26004253,
        0.26650803]])

In [9]:
#accuracy
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(pred,test):
    pred = pred[test.nonzero()].flatten()
    test = test[test.nonzero()].flatten()
    return sqrt(mean_squared_error(pred,test))

print(rmse(user_pred,test_data_matrix))
print(rmse(book_pred,test_data_matrix))

3.121195406647547
3.4462683737531132
